In [2]:
import cv2
import numpy as np
import os
import pickle

In [1]:
with open('processed_images.pickle', 'rb') as f:
    processed_images = pickle.load(f)
processed_images

NameError: name 'pickle' is not defined

In [ ]:
with open('window_labels.pickle', 'rb') as f:
    window_labels = pickle.load(f)
window_labels

In [45]:
def load_images_from_folder(folder):
    image_tuples = []
    for filename in os.listdir(folder):
        if (filename not in processed_images):
            img = cv2.imread(os.path.join(folder,filename))
            if (img is not None):
                image_tuples.append((filename, img))
    return image_tuples

In [46]:
def threshold_image(image, threshold=25):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    for i in range(len(gray)):
        for j in range(len(gray[i])):
            gray[i][j] = int((gray[i][j] > threshold) and not (image[i][j][0] < 200 and image[i][j][1] < 200 and image[i][j][2] > threshold)) * 255
    return gray

In [47]:
def generate_windows(image, filename, window_size, interior_size, increment, folder):
    x = 0
    while (x < image.shape[0] - window_size):
        row_has_intersection = False
        y = 0
        while (y < image.shape[1] - window_size):
            window = image[x : x + window_size, y : y + window_size, :]
            has_intersection = False
            
            gray = threshold_image(window)
            sum_of_grayscale_pixels = np.sum(gray) / 255
            
            # if the image is not completely white, we can add it to the dataset
            if (sum_of_grayscale_pixels < window_size * window_size):
                
                # check center interior square for any red pixels
                for x_interior in range (int(window_size/2 - interior_size/2), int(window_size/2 + interior_size/2)):
                    for y_interior in range (int(window_size/2 - interior_size/2), int(window_size/2 + interior_size/2)): 
                        # check if pixel is red
                        if (not has_intersection and 
                            window[x_interior][y_interior][0] < 100 and 
                            window[x_interior][y_interior][1] < 100 and
                            window[x_interior][y_interior][2] > 200):
                            row_has_intersection = True
                            has_intersection = True

                window_id = str(x) + "_" + str(y) + "_" + filename
                window_labels[window_id] = int(has_intersection)
                cv2.imwrite(os.path.join(folder, window_id), gray)
                
            if (has_intersection):
                y += int(increment / 5)
            else:
                y += increment
        if (row_has_intersection):
            x += int(increment / 5)
        else:
            x += increment

    processed_images.append(filename)

In [48]:
image_tuples = load_images_from_folder("images")
for image_tuple in image_tuples:
    filename = image_tuple[0]
    image = image_tuple[1]
    generate_windows(image=image, filename=filename, window_size=25, interior_size=5, increment=10, folder="dataset")

In [49]:
with open('processed_images.pickle', 'wb') as f:
    pickle.dump(processed_images, f)

In [50]:
with open('window_labels.pickle', 'wb') as f:
    pickle.dump(window_labels, f)

In [51]:
# processed_images = []
# window_labels = {}

In [52]:
window_labels

{'70_680_001.jpeg': 0,
 '70_690_001.jpeg': 0,
 '70_700_001.jpeg': 0,
 '70_710_001.jpeg': 0,
 '70_720_001.jpeg': 0,
 '80_450_001.jpeg': 0,
 '80_460_001.jpeg': 0,
 '80_470_001.jpeg': 0,
 '80_480_001.jpeg': 0,
 '80_490_001.jpeg': 0,
 '80_500_001.jpeg': 0,
 '80_510_001.jpeg': 0,
 '80_520_001.jpeg': 0,
 '80_530_001.jpeg': 0,
 '80_540_001.jpeg': 0,
 '80_550_001.jpeg': 0,
 '80_560_001.jpeg': 0,
 '80_570_001.jpeg': 0,
 '80_580_001.jpeg': 0,
 '80_590_001.jpeg': 0,
 '80_600_001.jpeg': 0,
 '80_610_001.jpeg': 0,
 '80_620_001.jpeg': 0,
 '80_630_001.jpeg': 0,
 '80_640_001.jpeg': 0,
 '80_650_001.jpeg': 0,
 '80_660_001.jpeg': 0,
 '80_670_001.jpeg': 0,
 '80_680_001.jpeg': 0,
 '80_690_001.jpeg': 0,
 '80_700_001.jpeg': 0,
 '80_710_001.jpeg': 1,
 '80_712_001.jpeg': 1,
 '80_714_001.jpeg': 0,
 '80_724_001.jpeg': 0,
 '82_430_001.jpeg': 0,
 '82_440_001.jpeg': 0,
 '82_450_001.jpeg': 0,
 '82_460_001.jpeg': 0,
 '82_470_001.jpeg': 0,
 '82_480_001.jpeg': 0,
 '82_490_001.jpeg': 0,
 '82_500_001.jpeg': 0,
 '82_510_00